In [1]:
!pip install torchtext==0.6

     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 10.4MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torchtext.vocab import GloVe
from torchtext.data import Field, BucketIterator
from torchtext.data.metrics import bleu_score
import spacy
import pickle

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
#Reading files

with open('/content/drive/My Drive/Assignment4_2/trainen.txt', encoding='utf8') as f:
    eng_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('/content/drive/My Drive/Assignment4_2/trainta.txt', encoding='utf8') as f:
    tamil_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('/content/drive/My Drive/Assignment4_2/deven.txt', encoding='utf8') as f:
    eng_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
with open('/content/drive/My Drive/Assignment4_2/devta.txt', encoding='utf8') as f:
    tamil_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
# with open('trainen.txt', encoding='utf8') as f:
#     eng_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
# with open('trainta.txt', encoding='utf8') as f:
#     tamil_train = list(map(lambda x: x.rstrip(), f.readlines()))
    
# with open('deven.txt', encoding='utf8') as f:
#     eng_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
# with open('devta.txt', encoding='utf8') as f:
#     tamil_test = list(map(lambda x: x.rstrip(), f.readlines()))
    
# embedding_glove = GloVe(name='6B', dim=100)
embedding_glove = pickle.load(open('/content/drive/My Drive/Assignment4_2/glove.sav', 'rb'))

spacy_en = spacy.load('en_core_web_sm')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

stop_words = [',','.','?','!',')','(',':',']','[','$','#','&','%','--']
ENG = Field(tokenize = tokenize_en, init_token='sos', eos_token = 'eos', lower=True, stop_words=stop_words)
processed_eng_train = list(map(lambda x: ENG.preprocess(x), eng_train))
processed_eng_test = list(map(lambda x: ENG.preprocess(x), eng_test))

ENG.build_vocab(processed_eng_train, vectors=embedding_glove)

In [6]:
def preprocess(processed_eng):
    
    #function to return the numericalized version of the tokenized sentences
    X = []
    for tokenized_sentence in processed_eng:
        int_sequence = [2]  #first element is the SOS token 
        for token in tokenized_sentence:
            int_sequence.append(ENG.vocab.stoi[token])
        int_sequence.append(3) #last element is the EOS token
        X.append(int_sequence)
    
    return X

# X_train and X_test are lists of lists with the integer sequences for a given sentence
X_train = preprocess(processed_eng_train)
X_test = preprocess(processed_eng_test)

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Same thing for Tamil sentences
TAM = Tokenizer()
TAM.fit_on_texts(tamil_train)
Y_train = TAM.texts_to_sequences(tamil_train)
Y_test = TAM.texts_to_sequences(tamil_test)

#adding EOS token
_ = [y.append(0) for y in Y_train]
_ = [y.append(0) for y in Y_test]

Using TensorFlow backend.


In [8]:
source_vocab_size = len(ENG.vocab)
target_vocab_size = len(TAM.word_index)+2
print(source_vocab_size)
print(target_vocab_size)

9737
18669


In [9]:
class Encoder(nn.Module):
    
    def __init__(self, embed_size, hidden_size, ENG):
        
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding.from_pretrained(ENG.vocab.vectors)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        
    def forward(self, x, enc_hidden):
        
        embedded_x = self.embed(x)  # Ts x 100
        enc_output, enc_hidden = self.lstm(embedded_x.unsqueeze(1), enc_hidden)
        return enc_output, enc_hidden

In [10]:
h1 = torch.rand(2, 4)
h2 = torch.rand(1, 1, 4)

print(h1)
print(h2)

tensor([[0.6880, 0.5860, 0.7212, 0.2278],
        [0.5532, 0.4187, 0.2648, 0.4213]])
tensor([[[0.8132, 0.7761, 0.3467, 0.0980]]])


In [16]:
torch.mm(h1, h2.squeeze().view(-1, 1)).shape

torch.Size([2, 1])

In [1]:
class AttnDecoder(nn.Module):
    
    def __init__(self, embed_size, hidden_size, n_classes):
        
        super(AttnDecoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(n_classes, embed_size)
        self.lstm = nn.LSTM(embed_size + hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, n_classes)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, enc_hidden_states):
        
        # hidden is a tuple with each element of size 1,1,self.hidden_size
        # enc_hidden_states is of size Ts, self.hidden_size (Ts is the source length)
        embedded = self.embed(input).view(1, 1, -1)   # size is 1,1,self.embed_size
        
        #attn_scores will be of shape Ts
        attn_scores = torch.mm(enc_hidden_states, hidden[0].squeeze().view(-1,1)).squeeze()

        attn_weights = F.softmax(attn_scores, dim=0)
        
        # context will be of shape 1,1,self.hidden_size
        context = torch.bmm(attn_weights.view(1,1,-1), enc_hidden_states.unsqueeze(0))
        
        # new_input will be of shape 1,self.hidden_size + self.embed_size
        new_input = torch.cat((embedded[0], context[0]), 1)
        
        output, hidden = self.lstm(new_input.unsqueeze(0), hidden)

        output = self.logsoftmax(self.fc(output[0]))
        
        return output, hidden

NameError: name 'nn' is not defined

In [ ]:
def loss_per_pair(source_sentence, target_sentence, enc_obj, dec_obj, enc_optimiser, dec_optimiser, loss_fn):
    
    Ts = source_sentence.size(-1)
    Tt = target_sentence.size(-1)
    enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))
    
    enc_optimiser.zero_grad()
    dec_optimiser.zero_grad()
    
    loss_val = 0
    
    enc_output, enc_hidden = enc_obj(source_sentence, enc_hidden) # enc_output will be Ts x 1 x hidden_size
        
    dec_input = torch.tensor([[target_vocab_size-1]], device=device)  #SOS token
    
    dec_hidden = enc_hidden
    
    for i in range(Tt):
        dec_output, dec_hidden = dec_obj(dec_input, dec_hidden, enc_output.squeeze(1))
        _ , index = dec_output.topk(1)
        dec_input = index.squeeze().detach()
        
        target_word = torch.cuda.LongTensor([target_sentence[i].item()])
        
        loss_val += loss_fn(dec_output, target_word)
        
        if dec_input.item() == 0:
            break

    loss_val.backward()
    
    nn.utils.clip_grad_norm_(enc_obj.parameters(), 0.5, 1)
    nn.utils.clip_grad_norm_(dec_obj.parameters(), 0.5, 1)

    enc_optimiser.step()
    dec_optimiser.step()

    return loss_val.item()/Tt

In [ ]:
def train_model(sources, targets, enc_obj, dec_obj):
    
    loss_fn = nn.NLLLoss()
    enc_optimiser = optim.SGD(enc_obj.parameters(), lr=0.001, momentum=0.9)
    dec_optimiser = optim.SGD(dec_obj.parameters(), lr=0.001, momentum=0.9)
    
    max_epochs = 200
    old_loss = np.inf
    indices = [i for i in range(len(sources))]
    
    for epoch in range(max_epochs):
        
        # shuffling the input data manually at the start of every new epoch
        np.random.shuffle(indices)
        sources = list(np.array(sources)[indices])
        targets = list(np.array(targets)[indices])
        
        running_loss = 0.0
        num_sentences = len(sources)
        
        for i in range(num_sentences):
            source_sentence = torch.cuda.LongTensor(sources[i])
            target_sentence = torch.cuda.LongTensor(targets[i])

            loss = loss_per_pair(source_sentence, target_sentence, enc_obj, dec_obj, enc_optimiser, dec_optimiser, loss_fn)
            running_loss += loss

            if i%int(num_sentences*0.1)==0:
                print("Epoch", epoch+1, ":", (i/int(num_sentences*0.1))*10,'% done')
                print("Current loss:", running_loss)
                
        print('\nEpoch', epoch+1,"\n")
        print("Encoder obj lstm wts sum=", torch.sum(enc_obj.lstm.weight_ih_l0))
        print("Encoder obj lstm bias sum=", torch.sum(enc_obj.lstm.bias_ih_l0))
        print('Decoder obj emb wts sum=', torch.sum(dec_obj.embed.weight.data))
        print('Decoder obj lstm wts sum=', torch.sum(dec_obj.lstm.weight_ih_l0.data))
        print('Decoder obj lstm bias sum=', torch.sum(dec_obj.lstm.bias_ih_l0.data))
        print('Decoder obj linear wts sum=', torch.sum(dec_obj.fc.weight.data))
        print('Decoder obj linear bias sum=', torch.sum(dec_obj.fc.bias.data))
        
        if abs(running_loss-old_loss)/running_loss < 1e-4:
            print('Converged')
            break
    
        old_loss = running_loss

    print("Finished Training")

In [ ]:
embedding_size = 100
hidden_size = 150
encoder = Encoder(embedding_size, hidden_size, ENG).to(device)
decoder = AttnDecoder(embedding_size, hidden_size, target_vocab_size).to(device)
train_model(X_train, Y_train, encoder, decoder)

Epoch 1 : 0.0 % done
Current loss: 9.761347770690918
Epoch 1 : 10.0 % done
Current loss: 25722.622101867826
Epoch 1 : 20.0 % done
Current loss: 51419.551730597865
Epoch 1 : 30.0 % done
Current loss: 76504.73394887363
Epoch 1 : 40.0 % done
Current loss: 100120.44813214155
Epoch 1 : 50.0 % done
Current loss: 114823.96439688418
Epoch 1 : 60.0 % done
Current loss: 123280.46197410204
Epoch 1 : 70.0 % done
Current loss: 130402.98479227147
Epoch 1 : 80.0 % done
Current loss: 137191.065316288
Epoch 1 : 90.0 % done
Current loss: 143567.85100204308
Epoch 1 : 100.0 % done
Current loss: 150183.98504863324

Epoch 1 

Encoder obj lstm wts sum= tensor(0.1122, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(-1.9066, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(140.1962, device='cuda:0')
Decoder obj lstm wts sum= tensor(12.5563, device='cuda:0')
Decoder obj lstm bias sum= tensor(0.3138, device='cuda:0')
Decoder obj linear wts sum= tensor(41.6170, d

Epoch 9 : 10.0 % done
Current loss: 12142.30834804117
Epoch 9 : 20.0 % done
Current loss: 24133.47028765388
Epoch 9 : 30.0 % done
Current loss: 35996.588912490406
Epoch 9 : 40.0 % done
Current loss: 47747.265418903255
Epoch 9 : 50.0 % done
Current loss: 59557.925468834364
Epoch 9 : 60.0 % done
Current loss: 71594.53875926104
Epoch 9 : 70.0 % done
Current loss: 83378.30471483692
Epoch 9 : 80.0 % done
Current loss: 95175.98767341355
Epoch 9 : 90.0 % done
Current loss: 106954.5158094883
Epoch 9 : 100.0 % done
Current loss: 119049.8598388047

Epoch 9 

Encoder obj lstm wts sum= tensor(-0.2642, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(-0.8055, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(140.1676, device='cuda:0')
Decoder obj lstm wts sum= tensor(9.9023, device='cuda:0')
Decoder obj lstm bias sum= tensor(0.9805, device='cuda:0')
Decoder obj linear wts sum= tensor(48.4120, device='cuda:0')
Decoder obj linear bias sum= tensor(3.120

Epoch 17 : 10.0 % done
Current loss: 10935.059991337153
Epoch 17 : 20.0 % done
Current loss: 22029.63481463565
Epoch 17 : 30.0 % done
Current loss: 33127.05674032132
Epoch 17 : 40.0 % done
Current loss: 44073.95222317476
Epoch 17 : 50.0 % done
Current loss: 54945.04811559472
Epoch 17 : 60.0 % done
Current loss: 65858.66695008826
Epoch 17 : 70.0 % done
Current loss: 76726.07772954898
Epoch 17 : 80.0 % done
Current loss: 87619.16218864774
Epoch 17 : 90.0 % done
Current loss: 98576.9920263841
Epoch 17 : 100.0 % done
Current loss: 109373.49738540538

Epoch 17 

Encoder obj lstm wts sum= tensor(-0.4278, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(2.2430, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(140.1029, device='cuda:0')
Decoder obj lstm wts sum= tensor(3.4732, device='cuda:0')
Decoder obj lstm bias sum= tensor(2.0692, device='cuda:0')
Decoder obj linear wts sum= tensor(57.6017, device='cuda:0')
Decoder obj linear bias sum= tens

Epoch 25 : 10.0 % done
Current loss: 10392.798434645842
Epoch 25 : 20.0 % done
Current loss: 20720.68125502302
Epoch 25 : 30.0 % done
Current loss: 31287.80938479594
Epoch 25 : 40.0 % done
Current loss: 41573.5407337252
Epoch 25 : 50.0 % done
Current loss: 51874.72294775939
Epoch 25 : 60.0 % done
Current loss: 62250.09809569443
Epoch 25 : 70.0 % done
Current loss: 72712.62505074195
Epoch 25 : 80.0 % done
Current loss: 83299.57021420595
Epoch 25 : 90.0 % done
Current loss: 93669.35398864654
Epoch 25 : 100.0 % done
Current loss: 104034.88737335507

Epoch 25 

Encoder obj lstm wts sum= tensor(0.2564, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(4.0768, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(140.0854, device='cuda:0')
Decoder obj lstm wts sum= tensor(-0.1163, device='cuda:0')
Decoder obj lstm bias sum= tensor(2.6534, device='cuda:0')
Decoder obj linear wts sum= tensor(66.2671, device='cuda:0')
Decoder obj linear bias sum= tens

Epoch 33 : 10.0 % done
Current loss: 10118.96367260125
Epoch 33 : 20.0 % done
Current loss: 20298.969199589497
Epoch 33 : 30.0 % done
Current loss: 30432.7217712188
Epoch 33 : 40.0 % done
Current loss: 40678.409640711165
Epoch 33 : 50.0 % done
Current loss: 50929.10334422874
Epoch 33 : 60.0 % done
Current loss: 61060.10765547092
Epoch 33 : 70.0 % done
Current loss: 71370.98267678246
Epoch 33 : 80.0 % done
Current loss: 81350.78070839854
Epoch 33 : 90.0 % done
Current loss: 91527.43659107918
Epoch 33 : 100.0 % done
Current loss: 101666.82966476776

Epoch 33 

Encoder obj lstm wts sum= tensor(0.9108, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(5.1941, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(140.0818, device='cuda:0')
Decoder obj lstm wts sum= tensor(-2.0522, device='cuda:0')
Decoder obj lstm bias sum= tensor(2.9807, device='cuda:0')
Decoder obj linear wts sum= tensor(74.1144, device='cuda:0')
Decoder obj linear bias sum= ten

Epoch 41 : 10.0 % done
Current loss: 10020.255986145476
Epoch 41 : 20.0 % done
Current loss: 20096.131479103577
Epoch 41 : 30.0 % done
Current loss: 29915.821240456466
Epoch 41 : 40.0 % done
Current loss: 39737.10901501328
Epoch 41 : 50.0 % done
Current loss: 49689.214455925605
Epoch 41 : 60.0 % done
Current loss: 59678.25657573501
Epoch 41 : 70.0 % done
Current loss: 69692.10418735711
Epoch 41 : 80.0 % done
Current loss: 79646.37814248222
Epoch 41 : 90.0 % done
Current loss: 89563.37193411554
Epoch 41 : 100.0 % done
Current loss: 99546.25603091357

Epoch 41 

Encoder obj lstm wts sum= tensor(1.5275, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(6.4156, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(140.0791, device='cuda:0')
Decoder obj lstm wts sum= tensor(-4.1376, device='cuda:0')
Decoder obj lstm bias sum= tensor(3.2920, device='cuda:0')
Decoder obj linear wts sum= tensor(81.4566, device='cuda:0')
Decoder obj linear bias sum= t

Epoch 49 : 10.0 % done
Current loss: 9801.248745864199
Epoch 49 : 20.0 % done
Current loss: 19642.74043678557
Epoch 49 : 30.0 % done
Current loss: 29376.211955497416
Epoch 49 : 40.0 % done
Current loss: 39176.82955294443
Epoch 49 : 50.0 % done
Current loss: 49080.86017850145
Epoch 49 : 60.0 % done
Current loss: 58718.12984761932
Epoch 49 : 70.0 % done
Current loss: 68503.33173622322
Epoch 49 : 80.0 % done
Current loss: 78176.02594586581
Epoch 49 : 90.0 % done
Current loss: 88111.64188876806
Epoch 49 : 100.0 % done
Current loss: 97997.75762222495

Epoch 49 

Encoder obj lstm wts sum= tensor(2.1411, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(7.3959, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(140.0777, device='cuda:0')
Decoder obj lstm wts sum= tensor(-5.8027, device='cuda:0')
Decoder obj lstm bias sum= tensor(3.5445, device='cuda:0')
Decoder obj linear wts sum= tensor(88.3593, device='cuda:0')
Decoder obj linear bias sum= tens

Epoch 57 : 10.0 % done
Current loss: 9733.493026733473
Epoch 57 : 20.0 % done
Current loss: 19485.258110865718
Epoch 57 : 30.0 % done
Current loss: 29149.544626443356
Epoch 57 : 40.0 % done
Current loss: 38853.21945679324
Epoch 57 : 50.0 % done
Current loss: 48515.557830558
Epoch 57 : 60.0 % done
Current loss: 58101.68755138644
Epoch 57 : 70.0 % done
Current loss: 67774.24351367801
Epoch 57 : 80.0 % done
Current loss: 77504.4135747647
Epoch 57 : 90.0 % done
Current loss: 87331.2054278973
Epoch 57 : 100.0 % done
Current loss: 97005.21350526497

Epoch 57 

Encoder obj lstm wts sum= tensor(2.6400, device='cuda:0', grad_fn=<SumBackward0>)
Encoder obj lstm bias sum= tensor(7.9909, device='cuda:0', grad_fn=<SumBackward0>)
Decoder obj emb wts sum= tensor(140.0758, device='cuda:0')
Decoder obj lstm wts sum= tensor(-6.8506, device='cuda:0')
Decoder obj lstm bias sum= tensor(3.7212, device='cuda:0')
Decoder obj linear wts sum= tensor(94.8169, device='cuda:0')
Decoder obj linear bias sum= tensor(

KeyboardInterrupt: 

In [ ]:
torch.save(encoder.state_dict(), 'q4_encoder_weights.pt')
torch.save(decoder.state_dict(), 'q4_decoder_weights.pt')

In [ ]:
def test_loss(sources, targets, enc_obj, dec_obj):
    
    loss_fn = nn.NLLLoss()
    num_sentences = len(sources)
    total_loss = 0
        
    for i in range(num_sentences):

        with torch.no_grad():

            source_sentence = torch.cuda.LongTensor(sources[i])
            target_sentence = torch.cuda.LongTensor(targets[i])
            Ts = source_sentence.size(-1)
            Tt = target_sentence.size(-1)
            enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))
            
            loss_val = 0
            
            enc_output, enc_hidden = enc_obj(source_sentence[i], enc_hidden)

            dec_input = torch.tensor([[target_vocab_size-1]], device=device)  #SOS token

            # first hidden state of decoder is made the final hidden state of the encoder
            dec_hidden = enc_hidden

            for i in range(Tt):

                dec_output, dec_hidden = dec_obj(dec_input, dec_hidden, enc_output.squeeze(1))
                _ , index = dec_output.topk(1)
                dec_input = index.squeeze().detach()
                if index.item() == 0:
                    break

                target_word = torch.cuda.LongTensor([target_sentence[i].item()])

                loss_val += loss_fn(dec_output, target_word)
            
            total_loss += loss_val.item()/Tt
    
    return total_loss

In [ ]:
# this test loss is for 500 sentences
# test_loss(X_test, Y_test, encoder, decoder)

In [ ]:
def eval_bleu(enc_obj, dec_obj, source_sentence, target, target_vocab_dict):
    
    # function to return the BLEU score for a single sentence 
    
    with torch.no_grad():
        
        Ts = source_sentence.size(-1)
        enc_hidden = (torch.zeros(1, 1, enc_obj.hidden_size, device=device), torch.zeros(1, 1, enc_obj.hidden_size, device=device))

        enc_output, enc_hidden = enc_obj(source_sentence, enc_hidden)

        dec_input = torch.tensor([[target_vocab_size-1]], device=device)  # SOS

        dec_hidden = enc_hidden
        
        predicted = []

        for i in range(60):
            dec_output, dec_hidden = dec_obj(dec_input, dec_hidden, enc_output.squeeze(1))
            _ , index = dec_output.data.topk(1)
            if index.item() == 0:
                #decoded_words.append('<EOS>')
                break
            else:
                predicted.append(target_vocab_dict[index.item()])

            dec_input = index.squeeze().detach()

#     print(predicted)
#     print(target)
    return bleu_score([predicted], [[target]])

In [ ]:
def evaluate_model(enc_obj, dec_obj, source_test, target_test, target_vocab_dict):
    
    # returns the average bleu score for the model with the given test data
    
    total_bleu = 0
    for i in range(len(source_test)):
        source_sentence = torch.cuda.LongTensor(source_test[i])
        target = [target_vocab_dict[x] for x in target_test[i][:-1]]
        bleu = eval_bleu(enc_obj, dec_obj, source_sentence, target, target_vocab_dict)
        total_bleu += bleu
    
    return total_bleu/len(source_test)

In [ ]:
evaluate_model(encoder, decoder, X_test, Y_test, TAM.index_word)

0.0